In [1]:
import sys
import datetime
from pathlib import Path
import pyarrow as pa
import pyarrow.parquet as pq
import os
import pandas as pd
import logging
sys.path.append(r'C:\Users\Administrator\finstral-1')
import qt_api.qt_api as qt
import src.local_symbols as local_symbols
import src.local_candles as local_candles


In [2]:
df = local_candles.load_parquet()
df

[WindowsPath('C:/Users/Administrator/finstral-1/data/parquet/20250406_AG_combined.parquet')]
📂 Loading most recent parquet file: 20250406_AG_combined.parquet


interval                  end    open    high  \
item_id timestamp                                                             
BRK.B   2025-01-06 06:15:00  OneMinute  2025-01-06 06:16:00  456.10  456.20   
        2025-01-06 06:21:00  OneMinute  2025-01-06 06:22:00  456.35  456.35   
        2025-01-06 07:00:00  OneMinute  2025-01-06 07:01:00  456.54  456.54   
        2025-01-06 07:59:00  OneMinute  2025-01-06 08:00:00  456.30  456.30   
        2025-01-06 08:00:00  OneMinute  2025-01-06 08:01:00  456.71  456.71   
...                                ...                  ...     ...     ...   
MDB     2025-04-04 18:54:00  OneMinute  2025-04-04 18:55:00  153.95  153.95   
        2025-04-04 19:00:00  OneMinute  2025-04-04 19:01:00  153.51  153.51   
        2025-04-04 19:36:00  OneMinute  2025-04-04 19:37:00  153.51  153.51   
        2025-04-04 19:50:00  OneMinute  2025-04-04 19:51:00  153.50  153.50   
        2025-04-04 19:59:00  OneMinute  2025-04-04 20:00:00  153.50  153.50   

                                low   close  volume        VWAP  \
item_id timestamp                                                 
BRK.B   2025-01-06 06:15:00  456.10  456.20     258  455.882852   
        2025-01-06 06:21:00  456.35  456.35     100  455.961142   
        2025-01-06 07:00:00  456.07  456.07     300  456.179291   
        2025-01-06 07:59:00  456.30  456.30     346  456.208096   
        2025-01-06 08:00:00  456.31  456.31    1484  456.240498   
...                             ...     ...     ...         ...   
MDB     2025-04-04 18:54:00  153.95  153.95     496  156.060306   
        2025-04-04 19:00:00  153.51  153.51     110  156.060191   
        2025-04-04 19:36:00  153.51  153.51     398  156.060250   
        2025-04-04 19:50:00  153.50  153.50     110  156.060318   
        2025-04-04 19:59:00  153.50  153.50     693  156.059773   

                                industrySector  
item_id timestamp                               
BRK.B   2025-01-06 06:15:00  FinancialServices  
        2025-01-06 06:21:00  FinancialServices  
        2025-01-06 07:00:00  FinancialServices  
        2025-01-06 07:59:00  FinancialServices  
        2025-01-06 08:00:00  FinancialServices  
...                                        ...  
MDB     2025-04-04 18:54:00         Technology  
        2025-04-04 19:00:00         Technology  
        2025-04-04 19:36:00         Technology  
        2025-04-04 19:50:00         Technology  
        2025-04-04 19:59:00         Technology  

[15765758 rows x 9 columns]

In [3]:
df['target'] = df['close'].astype(float)


In [4]:
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [6]:
predictor = TimeSeriesPredictor(
    prediction_length=48,
    path="../models/autogluon-close-minute",
    target="target",
    eval_metric="RMSE",
    freq="1min",
    verbosity=2,
    known_covariates_names=['industrySector', 'open', 'high', 'low', 'close', 'volume', 'VWAP']
)

# Enhanced hyperparameters leveraging your hardware
hyperparameters = {
    'DeepAR': {
        'epochs': 20,
        'learning_rate': 1e-3,
        'batch_size': 256,
        'hidden_size': 128,
        'num_layers': 4,
        'trainer_kwargs': {
            'accelerator': 'gpu',
            'devices': 1,
            'num_workers': 32
        }
    },
    'PatchTST': {
        'epochs': 20,
        'batch_size': 256,
        'patch_length': 12,
        'stride': 6,
        'num_layers': 3,
        'trainer_kwargs': {
            'accelerator': 'gpu',
            'devices': 1,
            'num_workers': 32
        }
    },
    'TemporalFusionTransformer': {
        'epochs': 20,
        'batch_size': 256,
        'hidden_size': 128,
        'attention_head_size': 4,
        'trainer_kwargs': {
            'accelerator': 'gpu',
            'devices': 1,
            'num_workers': 32
        }
    }
}

# Optimized fit configuration
predictor.fit(
    train_data=df,
    presets="fast_training",
    time_limit=1200,
    hyperparameters=hyperparameters,
    num_val_windows=3,
    refit_full=True,
    enable_ensemble=True,
    random_seed=123
)

Frequency '1min' stored as 'min'
Beginning AutoGluon training... Time limit = 1200s
AutoGluon will save models to 'c:\Users\Administrator\finstral-1\models\autogluon-close-minute'
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.9
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.17763
CPU Count:          40
GPU Count:          1
Memory Avail:       342.34 GB / 383.91 GB (89.2%)
Disk Space Avail:   563.19 GB / 1787.27 GB (31.5%)
Setting presets to: fast_training

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': RMSE,
 'freq': 'min',
 'hyperparameters': {'DeepAR': {'batch_size': 256,
                                'epochs': 20,
                                'hidden_size': 128,
                                'learning_rate': 0.001,
                                'num_layers': 4,
                                'trainer_kwargs': {'accelerator': 'gpu',
                                    

ValueError: Trainer has no fit models that can predict.

In [ ]:
predictions = predictor.predict(df)
predictions.head()